In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime
import time
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

current_time = datetime.now().strftime('%y-%m-%d')

# 로그인 정보 하드코딩
USERNAME = ''  # 여기에 Instagram 사용자 이름을 입력하세요
PASSWORD = ''  # 여기에 Instagram 비밀번호를 입력하세요

# 검색어 입력
search = input('검색어를 입력하세요: ')
search_input = search
search = urllib.parse.quote(search)
search_url = 'https://www.instagram.com/explore/tags/' + str(search) + '/'

# Chrome 드라이버 경로 설정
chrome_driver_path = 'C:\\chromedriver-win64\\chromedriver.exe' # 맞게 설정해줘야
service = Service(chrome_driver_path)

# Chrome 옵션 설정
options = Options()
options.add_argument("--disable-notifications")  # 알림 비활성화
options.add_argument("--start-maximized")  # 브라우저 창 최대화

# WebDriver 객체 생성
driver = webdriver.Chrome(service=service, options=options)

# Instagram 로그인 페이지로 이동
driver.get('https://www.instagram.com/accounts/login/')
time.sleep(2)  # 페이지 로드 대기

# 로그인 과정
username_input = driver.find_element(By.NAME, 'username')
password_input = driver.find_element(By.NAME, 'password')
login_button = driver.find_element(By.XPATH, '//button[@t="submit"]')

username_input.send_keys(USERNAME)
password_input.send_keys(PASSWORD)
login_button.click()

time.sleep(5)  # 로그인 후 대기

# 로그인 후 탐색
driver.get(search_url)
time.sleep(30)  # 페이지가 완전히 로드될 때까지 대기

# 스크롤하여 데이터를 수집하는 부분
SCROLL_PAUSE_TIME = 1
alt_texts = []  # alt 텍스트를 저장할 리스트

last_height = driver.execute_script('return document.body.scrollHeight')
while True:
    # 페이지 소스 가져오기
    pagestring = driver.page_source
    bs = BeautifulSoup(pagestring, 'lxml')
    
    # _aagv 클래스를 가진 모든 div 요소 찾기
    post_divs = bs.find_all('div', {'class': '_aagv'})
    
    for div in post_divs:
        img_tags = div.find_all('img')
        for img in img_tags:
            alt_text = img.get('alt')
            if alt_text:
                alt_texts.append(alt_text)
    
    if post_divs:
        print(f"Found {len(post_divs)} divs with class '_aagv' on this scroll.")
    
    # 페이지 스크롤
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script('return document.body.scrollHeight')
    
    # 스크롤이 끝났는지 확인
    if new_height == last_height:
        break
    last_height = new_height

# 결과 출력
if alt_texts:
    data = pd.DataFrame({
        "검색어": search_input,
        "검색결과": alt_texts
    })
    print(data)
else:
    print("No alt text found.")

data.to_excel(f"인스타그램크롤링 {current_time}.xlsx", index=False)

# 드라이버 종료
driver.quit()

검색어를 입력하세요:  검색어


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@t="submit"]"}
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF65E8F9632+30946]
	(No symbol) [0x00007FF65E8AE3C9]
	(No symbol) [0x00007FF65E7A6FDA]
	(No symbol) [0x00007FF65E7F822C]
	(No symbol) [0x00007FF65E7F850C]
	(No symbol) [0x00007FF65E83DCB7]
	(No symbol) [0x00007FF65E81CAAF]
	(No symbol) [0x00007FF65E83B041]
	(No symbol) [0x00007FF65E81C813]
	(No symbol) [0x00007FF65E7EA6E5]
	(No symbol) [0x00007FF65E7EB021]
	GetHandleVerifier [0x00007FF65EA2F83D+1301229]
	GetHandleVerifier [0x00007FF65EA3BDB7+1351783]
	GetHandleVerifier [0x00007FF65EA32A03+1313971]
	GetHandleVerifier [0x00007FF65E92DD06+245686]
	(No symbol) [0x00007FF65E8B758F]
	(No symbol) [0x00007FF65E8B3804]
	(No symbol) [0x00007FF65E8